# Data Munging

In [1]:
# Importing libraries 
import numpy as np 
import pandas as pd
import janitor

# !pip3 install pyjanitor # run this once

import sklearn 
from sklearn.impute import KNNImputer
from sklearn import preprocessing

In [2]:
data = pd.read_csv('../data/mushrooms.csv')

In [3]:
data.info() # We want to see the data types and how many null values do we have in the dataset.  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [4]:
data.columns # We want to clean column names automatically by replacing each - with _

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [5]:
data = data.clean_names()

In [6]:
data.columns # Check column names after automatic cleaning

Index(['class', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
       'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
       'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
       'stalk_surface_below_ring', 'stalk_color_above_ring',
       'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
       'ring_type', 'spore_print_color', 'population', 'habitat'],
      dtype='object')

In [7]:
data.describe()

,class,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [8]:
# We want to see what different values each column contains
# From here, we can see that the veil_type has one single value and therefore is redundant and not informative so we can proceed with dropping it 
data.columns.tolist()
for col in data.columns.tolist(): 
    print(col,':  ',data[col].unique())

class :   ['p' 'e']
cap_shape :   ['x' 'b' 's' 'f' 'k' 'c']
cap_surface :   ['s' 'y' 'f' 'g']
cap_color :   ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises :   ['t' 'f']
odor :   ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill_attachment :   ['f' 'a']
gill_spacing :   ['c' 'w']
gill_size :   ['n' 'b']
gill_color :   ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk_shape :   ['e' 't']
stalk_root :   ['e' 'c' 'b' 'r' '?']
stalk_surface_above_ring :   ['s' 'f' 'k' 'y']
stalk_surface_below_ring :   ['s' 'f' 'y' 'k']
stalk_color_above_ring :   ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
stalk_color_below_ring :   ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
veil_type :   ['p']
veil_color :   ['w' 'n' 'o' 'y']
ring_number :   ['o' 't' 'n']
ring_type :   ['p' 'e' 'l' 'f' 'n']
spore_print_color :   ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
population :   ['s' 'n' 'a' 'v' 'y' 'c']
habitat :   ['u' 'g' 'm' 'd' 'p' 'w' 'l']


In [9]:
data.drop('veil_type', axis = 1, inplace = True)

In [10]:
# We can see that columns have multiple repetitive values (letters), even though they mean different things in each column. 
# This might cause some problems like duplications when we create dummy variables for these categorical values. 

# We can also see that the column 'stalk_root' has a non-alphanumeric value and it might need some munging. 
# According to the dataset's documentation, the value '?' in stalk_root means that they are missing or unknown stalk root data. 
# Let's see how many of these missing values we have to decide if it'd be okay to drop these rows. 

vals = data['stalk_root'].value_counts().index.values.tolist()

NA_count = data['stalk_root'].value_counts().values

NA_frac = data['stalk_root'].value_counts().to_list()
NA_frac = [i/sum(NA_frac) for i in NA_frac]

pd.DataFrame(zip(NA_count,NA_frac), columns=['Count','Fraction'], index= vals)

# So, now we can see that if we drop the missing values in this column we're losing 30% of our data which accounts for about 2500 instances. 
# Dropping the rows is not the best solution in this case. 
# Therefore, we'll try to impute using KNN.
# Before that, the categorical value must be numerically encoded/labelled from 0 to n. 


# ['population', 'cap_shape', 'stalk_shape', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_below_ring']

,Count,Fraction
b,3776,0.464796
?,2480,0.305268
e,1120,0.137863
c,556,0.068439
r,192,0.023634


In [11]:
# Let's see the order of values in this column
data.stalk_root.unique()

array(['e', 'c', 'b', 'r', '?'], dtype=object)

In [12]:
le = preprocessing.LabelEncoder()

for i in data.columns.tolist():
    data[i]= le.fit_transform(data[i])

data

,class,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,2,5,5,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,2,5,5,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,2,5,5,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,2,1,7,7,2,1,0,7,4,2


In [13]:
# Order of values after encoding 
data['stalk_root'].unique()

array([3, 2, 1, 4, 0])

In [14]:
# But, for the models to impute the missing data, we should replace each 4 with a NaN. 
data.replace({'stalk_root': {0: np.nan}}).stalk_root.value_counts()

1.0    3776
3.0    1120
2.0     556
4.0     192
Name: stalk_root, dtype: int64

In [15]:
imputer = KNNImputer(missing_values = np.nan, n_neighbors=5, weights = 'distance')
imputer.fit_transform(data[['stalk_root']])

array([[3.],
       [2.],
       [2.],
       ...,
       [0.],
       [0.],
       [0.]])

In [16]:
data.stalk_root.value_counts()

1    3776
0    2480
3    1120
2     556
4     192
Name: stalk_root, dtype: int64

In [17]:
# We can see that KNNImputer didn't give us any useful results and we're again back on square 1. Therefore, we'll just drop the column. 
data.drop('stalk_root', axis= 1, inplace= True)

# DataViz

# Dimensionality Reduction

# Model Development